In [ ]:
%cd /content/drive/MyDrive/Sketch_RNN/Dataset/npy

[Errno 2] No such file or directory: '/content/drive/MyDrive/Sketch_RNN/Dataset/npy'
/content


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


efficientnet 설치

In [ ]:
!pip install efficientnet_pytorch

4/1AX4XfWi-H4TCA0PsjZglRYfofJ0QG-YTxzYDuKsABID45OLCe-2wWwQvSScCollecting efficientnet_pytorch
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-cp37-none-any.whl size=16443 sha256=ba725021fe9e559feb4f0cd13b4cdf1a6dda0c45fa973b0868b58e8afcb52e29
  Stored in directory: /root/.cache/pip/wheels/84/27/aa/c46d23c4e8cc72d41283862b1437e0b3ad318417e8ed7d5921
Successfully built efficientnet-pytorch


EfficientNet 테스트
 

In [ ]:
from PIL import Image
import torch
import numpy as np
from torchvision import transforms
from efficientnet_pytorch import EfficientNet

model = EfficientNet.from_pretrained('efficientnet-b0')

# Load ImageNet class names
import json
import matplotlib.pyplot as plt
labels_map = json.load(open('/content/drive/MyDrive/Sketch_RNN/Dataset/etc/label_map.txt'))
labels_map = [labels_map[str(i)] for i in range(1000)]

for i in range (10, 20):
  np_array = np.load('/content/drive/MyDrive/Sketch_RNN/Dataset/npy/full_numpy_bitmap_flower.npy')
  tmp = np_array[i].reshape(28, 28)
  added_dim = np.repeat(tmp[..., np.newaxis], 3, -1)
  new_img = Image.fromarray(added_dim)

  plt.imshow(new_img)

  # Preprocess image
  tfms = transforms.Compose([transforms.Resize(224), transforms.ToTensor(),
      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),])
  img = tfms(new_img).unsqueeze(0)
  print(img.shape) # torch.Size([1, 3, 224, 224])

  features = model.extract_features(img)
  print(features.shape) # torch.Size([1, 1280, 7, 7])


  # Classify
  model.eval()
  with torch.no_grad():
      outputs = model(img)

  # Print predictions
  print('-----')
  for idx in torch.topk(outputs, k=5).indices.squeeze(0).tolist():
      prob = torch.softmax(outputs, dim=1)[0, idx].item()
      print('{label:<75} ({p:.2f}%)'.format(label=labels_map[idx], p=prob*100))


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth



Loaded pretrained weights for efficientnet-b0


FileNotFoundError: ignored

Feature Extraction


In [ ]:
from efficientnet_pytorch import EfficientNet

from PIL import Image
import numpy as np
from torchvision import transforms

# 모델 생성
model = EfficientNet.from_pretrained('efficientnet-b0')


# ^^ 메인에서 로드하고 포문 내에서 extract_features 실행

def extract_feature(np_array, model):
  tmp = np_array.reshape(28, 28)
  added_dim = np.repeat(tmp[..., np.newaxis], 3, -1)
  new_img = Image.fromarray(added_dim)

  # plt.imshow(new_img)

  # Preprocess image
  tfms = transforms.Compose([transforms.Resize(224), transforms.ToTensor(),
      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),])
  img = tfms(new_img).unsqueeze(0)

  features = model.extract_features(img)
  features = features.flatten()
  return features

Loaded pretrained weights for efficientnet-b0


Clustering

In [ ]:
# clustering and dimension reduction
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

# for everything else
import os
import numpy as np
import matplotlib.pyplot as plt
from random import randint
import pandas as pd
import pickle
import torch
import json

In [ ]:
data = {}
label_num = 13
labels_map = json.load(open('/content/drive/MyDrive/Sketch_RNN/Dataset/etc/npy_label.txt'))
labels_map = [labels_map[str(i)] for i in range(label_num)] # t-shirt, smiley face 뺴고 진행

img_prefix = '/content/drive/MyDrive/Sketch_RNN/Dataset/npy/'
%cd /content/drive/MyDrive/Clustering

for i in range (len(labels_map)):
  img_loc = img_prefix + str(labels_map[i]) + '.npy'
  np_array = np.load(img_loc)
  # data_num = np_array.shape[0]
  data_num = 3000
  print(data_num)

  for j in range(data_num):
      features = extract_feature(np_array[j], model) # Tensor
      # Convert tensor to numpy array
      data[i * data_num + j] = features.cpu().detach().numpy()
  np.save('feature_2500', data) # 덮어쓰기
  print(str(i) + 'saved')

feat = np.array(list(data.values()))
# print(feat.shape)

pca = PCA(n_components=0.9, random_state = 22)
pca.fit(feat)
x = pca.transform(feat)

/content/drive/MyDrive/Clustering
3000
0saved
3000
1saved
3000
2saved
3000
3saved
3000
4saved
3000
5saved
3000
6saved
3000
7saved
3000
8saved
3000
9saved
3000
10saved
3000
11saved
3000
12saved


In [ ]:
print(x.shape)
%cd /content/drive/MyDrive/Clustering
print('hiasd')
!pwd

NameError: ignored

In [ ]:
kmeans_2000 = KMeans(n_clusters=len(labels_map),n_jobs=-1, random_state=22) # data 2000개
kmeans.fit(x)

NameError: ignored

In [ ]:
kmeans = KMeans(n_clusters=len(labels_map),n_jobs=-1, random_state=22) # data 200개
kmeans.fit(x)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=13, n_init=10, n_jobs=-1, precompute_distances='auto',
       random_state=22, tol=0.0001, verbose=0)

In [ ]:
filenames = np.arange(0, label_num * data_num)
# holds the cluster id and the images { id: [images] }
groups = {}
for file, cluster in zip(filenames,kmeans.labels_):
    if cluster not in groups.keys():
        groups[cluster] = []
        groups[cluster].append(file)
    else:
        groups[cluster].append(file)

In [ ]:
accu = []
sum = 0
cnt = 0


for i in range(label_num):
  length = len(groups[i])
  for j in range(length):
    if groups[i][j] >= i * data_num and groups[i][j] < (i+1)*data_num:
      cnt+=1
  accu.append( str(round(cnt / length * 100, 2)) + '%')

print(accu)



['29.85%', '32.46%', '28.91%', '40.76%', '76.47%', '75.78%', '37.4%', '73.13%', '65.77%', '53.85%', '53.8%', '54.05%', '28.81%']
